# Import

In [2]:
import pandas as pd
import numpy as np
import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
from konlpy.tag import Okt
from PIL import Image

from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

from nltk.corpus import reuters
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# 네이버 크롤링용 함수 생성

In [2]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    


# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}


#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

# 네이버 뉴스 크롤링 진행

In [ ]:
#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

In [ ]:
for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
        news_date = news_date[0:10]
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        news_date = news_date[0:10]
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

100%|██████████| 70/70 [00:21<00:00,  3.31it/s]

검색된 기사 갯수: 총  150 개
news_title:  70
news_url:  70
news_contents:  70
news_dates:  70


In [ ]:
#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
news_df.to_csv('{}.csv'.format(search),encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  61


In [3]:
import os

path = './메타버스_뉴스/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]

metaverse_news = pd.DataFrame()
for i in file_list_py:
    data = pd.read_csv(path + i)
    metaverse_news = metaverse_news.append(data)

C:\Users\coden\AppData\Local\Temp\ipykernel_12500\4088419958.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_12500\4088419958.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_12500\4088419958.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_12500\4088419958.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users

In [4]:
metaverse_news["date"]= metaverse_news["date"].str.replace(".", "-", )

C:\Users\coden\AppData\Local\Temp\ipykernel_12500\787848037.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metaverse_news["date"]= metaverse_news["date"].str.replace(".", "-", )


In [5]:
# date 기준 정렬해주기
metaverse_news = metaverse_news.sort_values('date')
metaverse_news= metaverse_news.reset_index()

In [6]:
# 불필요한 index열 제거
metaverse_news = metaverse_news.drop(['index'], axis = 1)

In [9]:
metaverse_news = metaverse_news.replace("None", np.nan, )
metaverse_news = metaverse_news.dropna(axis = 0)
metaverse_news.head()

date                           title  \
0  2022-10-11  팅크웨어, 中 체리차에 증강현실 내비게이션 솔루션 공급   
1  2022-10-11                  증강 현실로 먼저 입어보기   
2  2022-10-11          '증강 현실 기능을 통해 내가 아바타로'   
3  2022-10-11          '증강 현실 기능을 통해 내가 아바타로'   
4  2022-10-11                   증강 현실로 입어보는 옷   

                                                link  \
0  https://n.news.naver.com/mnews/article/029/000...   
1  https://n.news.naver.com/mnews/article/003/001...   
2  https://n.news.naver.com/mnews/article/003/001...   
3  https://n.news.naver.com/mnews/article/003/001...   
4  https://n.news.naver.com/mnews/article/003/001...   

                                             content  
0  [\n\t\t\t팅크웨어는 중국 완성차 기업인 체리자동차가 새로 선보이는 아리조 8...  
1  [\n\n\n\n\n[서울=뉴시스] 정병혁 기자 = 11일 서울 중구 동대문디자인플...  
2  [\n\n\n\n\n[서울=뉴시스] 정병혁 기자 = 11일 서울 중구 동대문디자인플...  
3  [\n\n\n\n\n[서울=뉴시스] 정병혁 기자 = 11일 서울 중구 동대문디자인플...  
4  [\n\n\n\n\n[서울=뉴시스] 정병혁 기자 = 11일 서울 중구 동대문디자인플...

# 빅카인즈 뉴스 크롤링

In [190]:
import os

path = './메타버스_뉴스_빅카인즈/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xlsx')]

metaverse_news = pd.DataFrame()
for i in file_list_py:
    data = pd.read_excel(path + i)
    data = data.iloc[:,0:3]
    metaverse_news = metaverse_news.append(data)

#중복 행 지우기
metaverse_news = metaverse_news.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(metaverse_news))

C:\Users\coden\AppData\Local\Temp\ipykernel_10744\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_10744\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_10744\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_10744\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)


중복 제거 후 행 개수:  31523


C:\Users\coden\AppData\Local\Temp\ipykernel_10744\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)


In [191]:
metaverse_news.head()

일자                                                 제목  \
0  20221112            걸핏하면 퇴사한다던 김 과장...그 얘기 쏙들어간 이유 [더테크웨이브]   
1  20221112              이승현 “미스코리아 진 당선, ‘고대생’ 덕도 있을 것” [인터뷰]   
2  20221112  나인아이, 'Young Boy' VFX 퍼포먼스 비디오 기습 공개! '다이내믹 퍼포먼스'   
3  20221112  치열해지는 반도체 경쟁 '최종병기 활'을 지켜라 [더 머니이스트-Dr.J’s Chi...   
4  20221111            [사설] 韓 美 간 의제된 北 사이버 위협 범국가적 컨트롤타워 시급하다   

                                                  본문  
0  테크업계 혹독한 ‘겨울의 시간’\n요즘 국내 테크 업계에선 혹독한 ‘겨울의 시간’이...  
1  수영복 심사 사자머리 다 사라져...변화 추구 \n정시로 고대 입학, 꾸준히 공부한...  
2  그룹 나인아이(NINE.i)가 신개념 퍼포먼스 비디오로 독창적인 행보를 이어가고 있...  
3  트럼프 정부 2년 간 중국과 무역 갈등을 빚었던 미국은 다시 무역 적자가 다시 늘어...  
4  제이크 설리번 미국 백악관 국가안보보좌관은 다음주 캄보디아 프놈펜에서의 아세안 주요...

In [192]:
print(min(metaverse_news['일자']))
print(max(metaverse_news['일자']))

20211112
20221112


In [193]:
metaverse_news.columns = ['date', 'title', 'content']

In [194]:
# date 기준 정렬해주기
metaverse_news = metaverse_news.sort_values('date')
metaverse_news = metaverse_news.reset_index()

In [196]:
# 불필요한 index열 제거
metaverse_news = metaverse_news.drop(['index'], axis = 1)

In [200]:
for i in range(len(metaverse_news.date)):
    metaverse_news.date.iloc[i] = str(metaverse_news.date.iloc[i])

C:\Users\coden\AppData\Local\Temp\ipykernel_10744\2331600958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metaverse_news.date.iloc[i] = str(metaverse_news.date.iloc[i])


In [201]:
metaverse_news.head()

date                                       title  \
0  20211112                 코인원, 연대 전략마케팅 학회와 산학협력 프로젝트   
1  20211112                         [시시비비]디지털 경쟁력 키울 정책   
2  20211112  “퀵커머스로 맛 본 편리함 못끊는다” [P-코노미가 온다 ③Platform]   
3  20211112          공공기관 교육에 메타버스? 근로복지공단, 가상 인재개발원 구축   
4  20211112       일부터 놀이까지 ‘컴투버스’에서 코엑스아쿠아리움, 디지털로 옮겨온다   

                                             content  
0  국내 암호화폐 거래소 코인원이 지난 1일부터 11일까지 코인원의 산업 경쟁력 강화를...  
1  ‘메타버스 채용 부스마다 구직자 북적’, ‘메타버스로 업무 보는 시대’ 최근에 많은...  
2  “처음에는 신종 코로나바이러스 감염증(코로나19)라는 환경적인 변화가 ‘퀵커머스’ ...  
3  [머니투데이 세종=오세중 기자] \n공공기관 대상 교육에도 메타버스 플랫폼이 도입된...  
4  컴투스(078340)는 계열사 위지윅스튜디오 얼반웍스가 서울오션아쿠아리움과 공동사업...

# 메타버스 관련주 등락률 가져오기

In [41]:
import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX')

In [50]:
from datetime import datetime
from pandas_datareader import data as pdr
import pandas as pd
import xlrd

start=datetime(2020,1,20)


samsung=pdr.DataReader('000660.KS','yahoo',start)

stock_code=['036570','289220','030520','047080','035420','190510',
            '171090','347860','094360','096630','054210']

krx=fdr.StockListing('KRX')



stock_name=krx[['Code','Name']]



stock_name1=stock_name[stock_name['Code'].isin(stock_code)]

stock_name1=stock_name1.reset_index()

stock_name1=stock_name1.drop(['index'],axis=1)


price=[]

for i in range(len(stock_name1)):
    price.append(fdr.DataReader('{}'.format(stock_name1['Code'][i]),'2021-11-12', '2022-11-12'))
    
    price[i].to_csv('{}.csv'.format(stock_name1['Name'][i]))


''' for i in range(len(stock_name1)):
    price.append(fdr.DataReader('{}'.format(stock_name1['Code'][i]),metaverse_news.date.iloc[0],metaverse_news.date.iloc[-1]))
    
    price[i].to_csv('{}.csv'.format(stock_name1['Name'][i]))
 '''

" for i in range(len(stock_name1)):\n    price.append(fdr.DataReader('{}'.format(stock_name1['Code'][i]),metaverse_news.date.iloc[0],metaverse_news.date.iloc[-1]))\n    \n    price[i].to_csv('{}.csv'.format(stock_name1['Name'][i]))\n "

In [11]:
# 저장된 파일 /메타버스_종목 경로에 옮겨준 뒤 실행하기

In [210]:
import os

path = './메타버스_종목/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]

df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_csv(path + i)[['Date', 'Change']]
    data.columns = ['date', i[:-4]]
    df = pd.concat([df,data], axis = 1)
    df = df.replace(" ", np.nan)
    df = df.replace("-", "")
    df = df.replace(0, np.nan)
    df = df.fillna(0)

metaverse_stock = df.iloc[:,1::2]
metaverse_stock.insert(loc = 0, column = 'date', value = data['date'])

In [211]:
for i in range(1,len(metaverse_stock.columns)):
    for j in range(len(metaverse_stock)):
        if metaverse_stock.iloc[j,i] <= 0:
            metaverse_stock.iloc[j,i] = 0
        
        else:
            metaverse_stock.iloc[j,i] = 1

metaverse_stock.head()

date  NAVER  나무가  선익시스템  알체라  에스코넥  엔씨소프트  이랜텍  자이언트스텝  칩스앤미디어  \
0  2021-11-12    1.0  1.0    1.0  0.0   1.0    0.0  1.0     1.0     1.0   
1  2021-11-15    1.0  1.0    1.0  1.0   1.0    0.0  1.0     1.0     1.0   
2  2021-11-16    0.0  1.0    1.0  1.0   0.0    1.0  0.0     1.0     1.0   
3  2021-11-17    0.0  1.0    1.0  1.0   1.0    1.0  1.0     1.0     0.0   
4  2021-11-18    1.0  1.0    1.0  0.0   1.0    1.0  1.0     1.0     1.0   

   한글과컴퓨터  한빛소프트  
0     1.0    0.0  
1     1.0    0.0  
2     1.0    1.0  
3     1.0    1.0  
4     1.0    0.0

In [212]:
# 뉴스 데이터와 맞게 date열 포맷
metaverse_stock["date"] = metaverse_stock["date"].str.replace("-", "", )

# 뉴스 감성분석 진행

In [101]:
print('학습데이터 전체 개수: {}'.format(len(metaverse_news)))

학습데이터 전체 개수: 31523


In [202]:
# title 
okt = Okt()
n_ = []
title_rename = []
for i in range(len(metaverse_news)):
  if(i % 10000 ==0):
    print(i,"단계 완료")
  title_rename.append(re.sub("[\(\[].*?[\)\]]", "",metaverse_news.iloc[i]['title']))
  n_.append(' '.join(okt.nouns(metaverse_news.iloc[i]['title'])))
metaverse_news['nouns_t'] = n_
metaverse_news['title']=title_rename
metaverse_news = metaverse_news[metaverse_news['nouns_t']!='']

#2차 불용어 제거
metaverse_news['title'] = metaverse_news['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
metaverse_news['title'].replace('', np.nan, inplace=True)
metaverse_news = metaverse_news.dropna(how='any')

0 단계 완료
10000 단계 완료
20000 단계 완료
30000 단계 완료


C:\Users\coden\AppData\Local\Temp\ipykernel_10744\1988881253.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  metaverse_news['title'] = metaverse_news['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [203]:
# content
okt = Okt()
n_ = []
content_rename = []
for i in range(len(metaverse_news)):
  if(i % 10000 ==0):
    print(i,"단계 완료")
  content_rename.append(re.sub("[\(\[].*?[\)\]]", "",metaverse_news.iloc[i]['content']))
  n_.append(' '.join(okt.nouns(metaverse_news.iloc[i]['content'])))
metaverse_news['nouns_c'] = n_
metaverse_news['content']=content_rename
metaverse_news = metaverse_news[metaverse_news['nouns_c']!='']

#2차 불용어 제거
metaverse_news['content'] = metaverse_news['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
metaverse_news['content'].replace('', np.nan, inplace=True)
metaverse_news = metaverse_news.dropna(how='any')

0 단계 완료
10000 단계 완료
20000 단계 완료
30000 단계 완료


C:\Users\coden\AppData\Local\Temp\ipykernel_10744\600717096.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  metaverse_news['content'] = metaverse_news['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [207]:
metaverse_news.head()

date                               title  \
0  20211112          코인원 연대 전략마케팅 학회와 산학협력 프로젝트   
1  20211112                       디지털 경쟁력 키울 정책   
2  20211112                 퀵커머스로 맛 본 편리함 못끊는다    
3  20211112    공공기관 교육에 메타버스 근로복지공단 가상 인재개발원 구축   
4  20211112  일부터 놀이까지 컴투버스에서 코엑스아쿠아리움 디지털로 옮겨온다   

                                             content  \
0  국내 암호화폐 거래소 코인원이 지난 일부터 일까지 코인원의 산업 경쟁력 강화를 주제...   
1  메타버스 채용 부스마다 구직자 북적 메타버스로 업무 보는 시대 최근에 많은 눈길을 ...   
2  처음에는 신종 코로나바이러스 감염증라는 환경적인 변화가 퀵커머스 도입을 가속화 했지...   
3   공공기관 대상 교육에도 메타버스 플랫폼이 도입된다 충북 진천군에 소재한 근로복지공...   
4  컴투스는 계열사 위지윅스튜디오 얼반웍스가 서울오션아쿠아리움과 공동사업양해각서를 체결...   

                              nouns_t  \
0          코인 연대 전략 마케팅 학회 산학 협력 프로젝트   
1                     시시비비 디지털 경쟁력 정책   
2                        퀵 커머스 맛 코 노미   
3  공공기관 교육 메타 버스 근로 복지 공단 가상 인재 개발 구축   
4           일 놀이 컴투 버스 코엑스 아쿠아리움 디지털로   

                                             nouns_c  
0  국내 암호 화폐 거래소 코인 지난 코인 산업 경쟁력 강화 주제 연세대 전략 마케팅 ...  
1  메타 버스 채용 부스 구직 북적 메타 버스 업무 시대 최근 눈길 끌 기사 제목 전 ...  
2  처음 신종 코로나바이러스 감염증 코로나 환경 변화 퀵 커머스 도입 가속 번 맛 소비...  
3  머니투데이 세종 중 기자 공공기관 대상 교육 메타 버스 플랫폼 도입 충북 진천군 소...  
4  컴투스 계 열사 위지윅 스튜디오 얼반웍스 오션 아쿠아리움 공동 사업 양해각서 를 체...

In [213]:
metaverse_stock.head()

date  NAVER  나무가  선익시스템  알체라  에스코넥  엔씨소프트  이랜텍  자이언트스텝  칩스앤미디어  한글과컴퓨터  \
0  20211112    1.0  1.0    1.0  0.0   1.0    0.0  1.0     1.0     1.0     1.0   
1  20211115    1.0  1.0    1.0  1.0   1.0    0.0  1.0     1.0     1.0     1.0   
2  20211116    0.0  1.0    1.0  1.0   0.0    1.0  0.0     1.0     1.0     1.0   
3  20211117    0.0  1.0    1.0  1.0   1.0    1.0  1.0     1.0     0.0     1.0   
4  20211118    1.0  1.0    1.0  0.0   1.0    1.0  1.0     1.0     1.0     1.0   

   한빛소프트  
0    0.0  
1    0.0  
2    1.0  
3    1.0  
4    0.0

In [222]:
final = pd.merge(metaverse_news,metaverse_stock, how = 'inner')

In [223]:
for i in range(6, len(final.columns)):
    print(final.iloc[:,i].value_counts())

0.0    14887
1.0    13423
Name: 나무가, dtype: int64
0.0    15911
1.0    12399
Name: 선익시스템, dtype: int64
0.0    15800
1.0    12510
Name: 알체라, dtype: int64
0.0    16134
1.0    12176
Name: 에스코넥, dtype: int64
0.0    15883
1.0    12427
Name: 엔씨소프트, dtype: int64
0.0    14763
1.0    13547
Name: 이랜텍, dtype: int64
0.0    15580
1.0    12730
Name: 자이언트스텝, dtype: int64
1.0    14174
0.0    14136
Name: 칩스앤미디어, dtype: int64
0.0    15764
1.0    12546
Name: 한글과컴퓨터, dtype: int64
0.0    15943
1.0    12367
Name: 한빛소프트, dtype: int64


# title 감성사전

In [228]:
# 나무가 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_나무가:' , len(vocab_t))

up = Counter(final.iloc[:]['나무가'])[1]
down = Counter(final.iloc[:]['나무가'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_나무가 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['나무가']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_나무가[noun] = vocab_t_나무가[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_나무가[noun] = vocab_t_나무가[noun] - up_ratio
# 선익시스템 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_선익시스템:' , len(vocab_t))

up = Counter(final.iloc[:]['선익시스템'])[1]
down = Counter(final.iloc[:]['선익시스템'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_선익시스템 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['선익시스템']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_선익시스템[noun] = vocab_t_선익시스템[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_선익시스템[noun] = vocab_t_선익시스템[noun] - up_ratio
# 알체라 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_알체라:' , len(vocab_t))

up = Counter(final.iloc[:]['알체라'])[1]
down = Counter(final.iloc[:]['알체라'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_알체라 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['알체라']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_알체라[noun] = vocab_t_알체라[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_알체라[noun] = vocab_t_알체라[noun] - up_ratio
# 에스코넥 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_에스코넥:' , len(vocab_t))

up = Counter(final.iloc[:]['에스코넥'])[1]
down = Counter(final.iloc[:]['에스코넥'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_에스코넥 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['에스코넥']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_에스코넥[noun] = vocab_t_에스코넥[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_에스코넥[noun] = vocab_t_에스코넥[noun] - up_ratio
# 엔씨소프트 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_엔씨소프트:' , len(vocab_t))

up = Counter(final.iloc[:]['엔씨소프트'])[1]
down = Counter(final.iloc[:]['엔씨소프트'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_엔씨소프트 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['엔씨소프트']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_엔씨소프트[noun] = vocab_t_엔씨소프트[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_엔씨소프트[noun] = vocab_t_엔씨소프트[noun] - up_ratio
# 이랜텍 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_이랜텍:' , len(vocab_t))

up = Counter(final.iloc[:]['이랜텍'])[1]
down = Counter(final.iloc[:]['이랜텍'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_이랜텍 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['이랜텍']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_이랜텍[noun] = vocab_t_이랜텍[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_이랜텍[noun] = vocab_t_이랜텍[noun] - up_ratio
# 자이언트스텝 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_자이언트스텝:' , len(vocab_t))

up = Counter(final.iloc[:]['자이언트스텝'])[1]
down = Counter(final.iloc[:]['자이언트스텝'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_자이언트스텝 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['자이언트스텝']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_자이언트스텝[noun] = vocab_t_자이언트스텝[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_자이언트스텝[noun] = vocab_t_자이언트스텝[noun] - up_ratio
# 칩스앤미디어 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_칩스앤미디어:' , len(vocab_t))

up = Counter(final.iloc[:]['칩스앤미디어'])[1]
down = Counter(final.iloc[:]['칩스앤미디어'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_칩스앤미디어 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['칩스앤미디어']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_칩스앤미디어[noun] = vocab_t_칩스앤미디어[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_칩스앤미디어[noun] = vocab_t_칩스앤미디어[noun] - up_ratio 
# 한글과컴퓨터 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_한글과컴퓨터:' , len(vocab_t))

up = Counter(final.iloc[:]['한글과컴퓨터'])[1]
down = Counter(final.iloc[:]['한글과컴퓨터'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_한글과컴퓨터 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['한글과컴퓨터']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_한글과컴퓨터[noun] = vocab_t_한글과컴퓨터[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_한글과컴퓨터[noun] = vocab_t_한글과컴퓨터[noun] - up_ratio
# 한빛소프트 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_한빛소프트:' , len(vocab_t))

up = Counter(final.iloc[:]['한빛소프트'])[1]
down = Counter(final.iloc[:]['한빛소프트'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_한빛소프트 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['한빛소프트']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_한빛소프트[noun] = vocab_t_한빛소프트[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_한빛소프트[noun] = vocab_t_한빛소프트[noun] - up_ratio
# NAVER =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_NAVER:' , len(vocab_t))

up = Counter(final.iloc[:]['NAVER'])[1]
down = Counter(final.iloc[:]['NAVER'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_NAVER = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['NAVER']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_NAVER[noun] = vocab_t_NAVER[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_NAVER[noun] = vocab_t_NAVER[noun] - up_ratio

title_나무가: 14220
title_선익시스템: 14220
title_알체라: 14220
title_에스코넥: 14220
title_엔씨소프트: 14220
title_이랜텍: 14220
title_자이언트스텝: 14220
title_칩스앤미디어: 14220
title_한글과컴퓨터: 14220
title_한빛소프트: 14220
title_NAVER: 14220


# content 감성사전

In [229]:
# 나무가 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_나무가:' , len(vocab_c))

up = Counter(final.iloc[:]['나무가'])[1]
down = Counter(final.iloc[:]['나무가'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_나무가 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['나무가']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_나무가[noun] = vocab_c_나무가[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_나무가[noun] = vocab_c_나무가[noun] - up_ratio
# 선익시스템 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_선익시스템:' , len(vocab_c))

up = Counter(final.iloc[:]['선익시스템'])[1]
down = Counter(final.iloc[:]['선익시스템'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_선익시스템 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['선익시스템']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_선익시스템[noun] = vocab_c_선익시스템[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_선익시스템[noun] = vocab_c_선익시스템[noun] - up_ratio
# 알체라 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_알체라:' , len(vocab_c))

up = Counter(final.iloc[:]['알체라'])[1]
down = Counter(final.iloc[:]['알체라'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_알체라 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['알체라']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_알체라[noun] = vocab_c_알체라[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_알체라[noun] = vocab_c_알체라[noun] - up_ratio
# 에스코넥 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_에스코넥:' , len(vocab_c))

up = Counter(final.iloc[:]['에스코넥'])[1]
down = Counter(final.iloc[:]['에스코넥'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_에스코넥 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['에스코넥']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_에스코넥[noun] = vocab_c_에스코넥[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_에스코넥[noun] = vocab_c_에스코넥[noun] - up_ratio
# 엔씨소프트 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_엔씨소프트:' , len(vocab_c))

up = Counter(final.iloc[:]['엔씨소프트'])[1]
down = Counter(final.iloc[:]['엔씨소프트'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_엔씨소프트 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['엔씨소프트']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_엔씨소프트[noun] = vocab_c_엔씨소프트[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_엔씨소프트[noun] = vocab_c_엔씨소프트[noun] - up_ratio
# 이랜텍 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_이랜텍:' , len(vocab_c))

up = Counter(final.iloc[:]['이랜텍'])[1]
down = Counter(final.iloc[:]['이랜텍'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_이랜텍 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['이랜텍']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_이랜텍[noun] = vocab_c_이랜텍[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_이랜텍[noun] = vocab_c_이랜텍[noun] - up_ratio
# 자이언트스텝 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_자이언트스텝:' , len(vocab_c))

up = Counter(final.iloc[:]['자이언트스텝'])[1]
down = Counter(final.iloc[:]['자이언트스텝'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_자이언트스텝 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['자이언트스텝']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_자이언트스텝[noun] = vocab_c_자이언트스텝[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_자이언트스텝[noun] = vocab_c_자이언트스텝[noun] - up_ratio
# 칩스앤미디어 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_칩스앤미디어:' , len(vocab_c))

up = Counter(final.iloc[:]['칩스앤미디어'])[1]
down = Counter(final.iloc[:]['칩스앤미디어'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_칩스앤미디어 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['칩스앤미디어']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_칩스앤미디어[noun] = vocab_c_칩스앤미디어[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_칩스앤미디어[noun] = vocab_c_칩스앤미디어[noun] - up_ratio 
# 한글과컴퓨터 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_한글과컴퓨터:' , len(vocab_c))

up = Counter(final.iloc[:]['한글과컴퓨터'])[1]
down = Counter(final.iloc[:]['한글과컴퓨터'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_한글과컴퓨터 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['한글과컴퓨터']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_한글과컴퓨터[noun] = vocab_c_한글과컴퓨터[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_한글과컴퓨터[noun] = vocab_c_한글과컴퓨터[noun] - up_ratio
# 한빛소프트 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_빛소프트:' , len(vocab_c))

up = Counter(final.iloc[:]['한빛소프트'])[1]
down = Counter(final.iloc[:]['한빛소프트'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_한빛소프트 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['한빛소프트']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_한빛소프트[noun] = vocab_c_한빛소프트[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_한빛소프트[noun] = vocab_c_한빛소프트[noun] - up_ratio
# NAVER =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_NAVER:' , len(vocab_c))

up = Counter(final.iloc[:]['NAVER'])[1]
down = Counter(final.iloc[:]['NAVER'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_NAVER = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['NAVER']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_NAVER[noun] = vocab_c_NAVER[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_NAVER[noun] = vocab_c_NAVER[noun] - up_ratio

content_나무가: 25814
content_선익시스템: 25814
content_알체라: 25814
content_에스코넥: 25814
content_엔씨소프트: 25814
content_이랜텍: 25814
content_자이언트스텝: 25814
content_칩스앤미디어: 25814
content_한글과컴퓨터: 25814
content_빛소프트: 25814
content_NAVER: 25814


# title 기반 감성스코어 연산 

In [230]:
# 나무가 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_나무가):
          continue
        else:
          sent_score = sent_score + vocab_t_나무가[j]
    total.append(sent_score/len(w))
    
final['title_score_나무가'] = total

# 선익시스템 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_선익시스템):
          continue
        else:
          sent_score = sent_score + vocab_t_선익시스템[j]
    total.append(sent_score/len(w))
    
final['title_score_선익시스템'] = total

# 알체라 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_알체라):
          continue
        else:
          sent_score = sent_score + vocab_t_알체라[j]
    total.append(sent_score/len(w))
    
final['title_score_알체라'] = total

# 에스코넥 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_에스코넥):
          continue
        else:
          sent_score = sent_score + vocab_t_에스코넥[j]
    total.append(sent_score/len(w))
    
final['title_score_에스코넥'] = total

# 엔씨소프트 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_엔씨소프트):
          continue
        else:
          sent_score = sent_score + vocab_t_엔씨소프트[j]
    total.append(sent_score/len(w))
    
final['title_score_엔씨소프트'] = total

# 이랜텍 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_이랜텍):
          continue
        else:
          sent_score = sent_score + vocab_t_이랜텍[j]
    total.append(sent_score/len(w))
    
final['title_score_이랜텍'] = total

# 자이언트스텝 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_자이언트스텝):
          continue
        else:
          sent_score = sent_score + vocab_t_자이언트스텝[j]
    total.append(sent_score/len(w))
    
final['title_score_자이언트스텝'] = total

# 칩스앤미디어 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_칩스앤미디어):
          continue
        else:
          sent_score = sent_score + vocab_t_칩스앤미디어[j]
    total.append(sent_score/len(w))
    
final['title_score_칩스앤미디어'] = total

# 한글과컴퓨터 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_한글과컴퓨터):
          continue
        else:
          sent_score = sent_score + vocab_t_한글과컴퓨터[j]
    total.append(sent_score/len(w))
    
final['title_score_한글과컴퓨터'] = total

# 한빛소프트 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_한빛소프트):
          continue
        else:
          sent_score = sent_score + vocab_t_한빛소프트[j]
    total.append(sent_score/len(w))
    
final['title_score_한빛소프트'] = total

# NAVER =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_NAVER):
          continue
        else:
          sent_score = sent_score + vocab_t_NAVER[j]
    total.append(sent_score/len(w))
    
final['title_score_NAVER'] = total

# content 기반 감성스코어 연산

In [231]:
# 나무가 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_나무가):
          continue
        else:
          sent_score = sent_score + vocab_c_나무가[j]
    total.append(sent_score/len(w))
    
final['content_score_나무가'] = total

# 선익시스템 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_선익시스템):
          continue
        else:
          sent_score = sent_score + vocab_c_선익시스템[j]
    total.append(sent_score/len(w))
    
final['content_score_선익시스템'] = total

# 알체라 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_알체라):
          continue
        else:
          sent_score = sent_score + vocab_c_알체라[j]
    total.append(sent_score/len(w))
    
final['content_score_알체라'] = total

# 에스코넥 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_에스코넥):
          continue
        else:
          sent_score = sent_score + vocab_c_에스코넥[j]
    total.append(sent_score/len(w))
    
final['content_score_에스코넥'] = total

# 엔씨소프트 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_엔씨소프트):
          continue
        else:
          sent_score = sent_score + vocab_c_엔씨소프트[j]
    total.append(sent_score/len(w))
    
final['content_score_엔씨소프트'] = total

# 이랜텍 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_이랜텍):
          continue
        else:
          sent_score = sent_score + vocab_c_이랜텍[j]
    total.append(sent_score/len(w))
    
final['content_score_이랜텍'] = total

# 자이언트스텝 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_자이언트스텝):
          continue
        else:
          sent_score = sent_score + vocab_c_자이언트스텝[j]
    total.append(sent_score/len(w))
    
final['content_score_자이언트스텝'] = total

# 칩스앤미디어 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_칩스앤미디어):
          continue
        else:
          sent_score = sent_score + vocab_c_칩스앤미디어[j]
    total.append(sent_score/len(w))
    
final['content_score_칩스앤미디어'] = total

# 한글과컴퓨터 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_한글과컴퓨터):
          continue
        else:
          sent_score = sent_score + vocab_c_한글과컴퓨터[j]
    total.append(sent_score/len(w))
    
final['content_score_한글과컴퓨터'] = total

# 한빛소프트 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_한빛소프트):
          continue
        else:
          sent_score = sent_score + vocab_c_한빛소프트[j]
    total.append(sent_score/len(w))
    
final['content_score_한빛소프트'] = total

# NAVER =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_NAVER):
          continue
        else:
          sent_score = sent_score + vocab_c_NAVER[j]
    total.append(sent_score/len(w))
    
final['content_score_NAVER'] = total

# score 확인

In [233]:
final.to_csv("final_vocab.csv", index=False, encoding="utf-8-sig")